# Basic Technical

In [25]:
import os
import glob
import shutil
import cv2
from ultralytics import YOLO

# Paths
INPUT_DIR = './finish'
CROP_DIR = './crop_finish'
SKIP_DIR = './skip_finish'

# Create output directories if they don't exist
os.makedirs(CROP_DIR, exist_ok=True)
os.makedirs(SKIP_DIR, exist_ok=True)

# Load your license plate detector model
yolo_det = YOLO('./models/yolo11_anpr_ghd.pt')

# Confidence threshold (0.0–1.0)
CONF_THRESH = 0.8

# Iterate over all JPGs in the input folder
for img_path in glob.glob(os.path.join(INPUT_DIR, '*.jpg')):
    img = cv2.imread(img_path)
    if img is None:
        continue

    # Run detection with confidence threshold
    results = yolo_det(img, conf=CONF_THRESH)

    # Collect all boxes with conf >= threshold
    boxes = []
    for res in results:
        for box, conf in zip(res.boxes.xyxy, res.boxes.conf):
            if conf >= CONF_THRESH:
                x1, y1, x2, y2 = map(int, box.cpu().numpy())
                boxes.append((x1, y1, x2, y2))

    base_name = os.path.splitext(os.path.basename(img_path))[0]
    if boxes:
        # Save each cropped plate
        for idx, (x1, y1, x2, y2) in enumerate(boxes, start=1):
            crop = img[y1:y2, x1:x2]
            out_path = os.path.join(CROP_DIR, f"{base_name}_plate{idx}.jpg")
            cv2.imwrite(out_path, crop)
    else:
        # No detections above threshold: move original to skip folder
        shutil.copy(img_path, os.path.join(SKIP_DIR, os.path.basename(img_path)))

print("Done processing all images.")


0: 640x384 1 LicensePlate, 39.4ms
Speed: 2.2ms preprocess, 39.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 LicensePlate, 30.6ms
Speed: 1.0ms preprocess, 30.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 LicensePlate, 28.9ms
Speed: 1.0ms preprocess, 28.9ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 LicensePlate, 24.7ms
Speed: 1.7ms preprocess, 24.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 LicensePlate, 27.1ms
Speed: 0.9ms preprocess, 27.1ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 LicensePlate, 27.7ms
Speed: 1.3ms preprocess, 27.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 LicensePlate, 106.9ms
Speed: 1.8ms preprocess, 106.9ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 LicensePlate, 107.2ms
Speed: 2.4ms preprocess, 107.2ms infer

# Filter technical

In [18]:
import os
import glob
import shutil
import cv2
import numpy as np
from ultralytics import YOLO
import math

# --- Paths & Params ---
INPUT_DIR    = './s3'
CROP_DIR     = './crop_image'
SKIP_DIR     = './skip_crop_image'
CONF_THRESH  = 0.8     # YOLO confidence threshold
FLAT_SIGMA   = 15      # sigma for flat-field
DENOISE_H    = 3       # h parameter for NLMeans
TEMPLATE_SZ  = (7,7)   # (templateWindowSize, searchWindowSize)

# Create output folders if needed
os.makedirs(CROP_DIR, exist_ok=True)
os.makedirs(SKIP_DIR, exist_ok=True)

# Load detector
yolo_det = YOLO('./models/license_plate_detector.pt')

# flat‐field function
def imflatfield(I, sigma):
    A = I.astype(np.float32) / 255.0
    Ihsv = cv2.cvtColor(A, cv2.COLOR_BGR2HSV)
    V = Ihsv[:, :, 2]
    filterSize = int(2 * np.ceil(2 * sigma) + 1)
    shading = cv2.GaussianBlur(V, (filterSize, filterSize), sigma, borderType=cv2.BORDER_REFLECT)
    meanV = np.mean(V)
    shading = np.maximum(shading, 1e-6)
    Vcorr = V * meanV / shading
    Ihsv[:, :, 2] = Vcorr
    B = cv2.cvtColor(Ihsv, cv2.COLOR_HSV2BGR)
    return np.round(np.clip(B * 255, 0, 255)).astype(np.uint8)

# Process each image
for img_path in glob.glob(os.path.join(INPUT_DIR, '*.jpg')):
    img = cv2.imread(img_path)
    if img is None:
        continue

    # 1. Detect plates
    results = yolo_det(img, conf=CONF_THRESH)[0]
    boxes = []
    for box, conf in zip(results.boxes.xyxy, results.boxes.conf):
        if conf >= CONF_THRESH:
            x1, y1, x2, y2 = map(int, box.cpu().numpy())
            boxes.append((x1, y1, x2, y2))

    base = os.path.splitext(os.path.basename(img_path))[0]
    if not boxes:
        # no plate → move to skip
        shutil.copy(img_path, os.path.join(SKIP_DIR, os.path.basename(img_path)))
        print(f"✖ {base}: no plate found, moved to skip.")
        continue

    # 2. For each detected plate: crop + flatfield + denoise + save
    for idx, (x1, y1, x2, y2) in enumerate(boxes, start=1):
        crop = img[y1:y2, x1:x2]
        # flat-field correction
        flat = imflatfield(crop, FLAT_SIGMA)
        # NLMeans denoise (color)
        denoised = cv2.fastNlMeansDenoisingColored(
            flat, None,
            h=DENOISE_H, hColor=DENOISE_H,
            templateWindowSize=TEMPLATE_SZ[0],
            searchWindowSize=TEMPLATE_SZ[1]
        )
        # save
        out_name = f"{base}_plate{idx}.jpg"
        cv2.imwrite(os.path.join(CROP_DIR, out_name), denoised)
        print(f"✔ {base}_plate{idx}: cropped & preprocessed.")

print("Done processing all images.")



0: 640x384 (no detections), 38.7ms
Speed: 2.6ms preprocess, 38.7ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)
✖ 1744089859416: no plate found, moved to skip.

0: 640x384 (no detections), 24.0ms
Speed: 2.2ms preprocess, 24.0ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)
✖ 1741078372064: no plate found, moved to skip.

0: 640x384 (no detections), 18.5ms
Speed: 2.0ms preprocess, 18.5ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 384)
✖ 1743477981295: no plate found, moved to skip.

0: 640x384 (no detections), 20.7ms
Speed: 0.8ms preprocess, 20.7ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)
✖ 1743719542977: no plate found, moved to skip.

0: 640x384 (no detections), 34.6ms
Speed: 0.8ms preprocess, 34.6ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 384)
✖ 1744380514801: no plate found, moved to skip.

0: 640x384 1 license_plate, 20.2ms
Speed: 0.8ms preprocess, 20.2ms inference, 0.3ms postproces

# Plate Straight

In [21]:
import pandas as pd
df = pd.read_csv('test.txt', sep=',', encoding='utf-8-sig')
df['file_name']     = './crop_image/' + df['file_name']
df['expected_full'] = df['no'] + df['province']